In [1]:
import numpy as np

num_samples = 217636
x_shape = (100, 200, 3)
y_shape = (8,)

x_dtype = 'float32'  # Determine the appropriate dtype
y_dtype = 'float32'  # Determine the appropriate dtype

# x_memmap = np.memmap('x_dataset_head_pos.memmap', dtype=x_dtype, mode='w+', shape=(num_samples,) + x_shape)
# y_memmap = np.memmap('y_dataset_head_pos.memmap', dtype=y_dtype, mode='w+', shape=(num_samples,) + y_shape)

In [17]:
# load x_memmap and y_memmap
x_memmap = np.memmap('x_dataset_head_pos.memmap', dtype=x_dtype, mode='r+', shape=(num_samples,) + x_shape)
y_memmap = np.memmap('y_dataset_head_pos.memmap', dtype=y_dtype, mode='r+', shape=(num_samples,) + y_shape)

In [26]:
import pickle
import glob
import numpy as np

def process_and_combine_pkl_files_to_memmap(directory_path, x_memmap, y_memmap):
    current_index = 0
    
    for file_path in glob.glob(directory_path + '/*.pkl'):
        print(f"Processing file: {file_path}")
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            
        if isinstance(data, dict) and 'X' in data and 'Y' in data:
            X_data = data['X']
            Y_data = data['Y']
            
            # Flatten Y_data
            flattened_Y_data = []
            for y in Y_data:
                # Make sure to handle both cases where y[0] could be a list or a numpy array
                gaze_data = np.array(y[0], dtype=np.float32) if isinstance(y[0], list) else y[0].astype(np.float32)
                head_pose_data = y[1].astype(np.float32)
                flattened_y = np.concatenate([gaze_data, head_pose_data])
                flattened_Y_data.append(flattened_y)
                
            Y_data = np.array(flattened_Y_data, dtype=np.float32)
        
        else:
            Y_numeric = []  # Initialize an empty list to hold the processed Y data
            for y in data[1]:
                numeric_values = y[:2] + y[3:]  # Adjusted to exclude index 3
                Y_numeric.append([float(val) for val in numeric_values])  # Convert to float
            
            Y_data = np.array(Y_numeric, dtype=np.float32)  # Convert the list to a numpy array of type float32
            X_data = np.array(data[0], dtype=np.float32)  # Ensure X_data is also properly formatted

        num_samples_in_file = len(X_data)
        x_batch = np.array(X_data, dtype=x_memmap.dtype).reshape((num_samples_in_file,) + x_shape)
        
        # No need to reshape Y_data as it is already in the correct shape after flattening
        y_batch = Y_data  # It should already be in the correct shape
        
        # Ensure we do not exceed the allocated memmap size
        if current_index + num_samples_in_file > len(x_memmap):
            raise ValueError("The dataset is larger than expected.")
        
        # Write directly to the memmap files
        x_memmap[current_index:current_index + num_samples_in_file] = x_batch
        y_memmap[current_index:current_index + num_samples_in_file] = y_batch
        
        current_index += num_samples_in_file
        x_memmap.flush()
        y_memmap.flush()

In [ ]:
directory_path = './process_MPIIGaze/batches_head_pos/' 
process_and_combine_pkl_files_to_memmap(directory_path, x_memmap, y_memmap)

In [17]:
def memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, indices, shuffle=True):
    x_memmap = np.memmap(x_memmap_path, dtype=x_dtype, mode='r', shape=(num_samples,) + x_shape)
    y_memmap = np.memmap(y_memmap_path, dtype=y_dtype, mode='r', shape=(num_samples,) + y_shape)
    
    if shuffle:
            np.random.shuffle(indices)
        
    while True:
        for start_idx in range(0, len(indices), batch_size):
            end_idx = min(start_idx + batch_size, len(indices))
            batch_indices = indices[start_idx:end_idx]
            
            # Yield a batch of data
            yield x_memmap[batch_indices], y_memmap[batch_indices]

In [18]:
batch_size = 32

In [19]:
# Paths to the memmap files
x_memmap_path = 'x_dataset_head_pos.memmap'
y_memmap_path = 'y_dataset_head_pos.memmap'

In [11]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Input layer
input_layer = Input(shape=(100, 200, 3))

# Convolutional layers
x = Conv2D(32, (7, 7), activation='relu', kernel_regularizer=l2(0.001))(input_layer)
x = Conv2D(64, (7, 7), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Dropout(0.15)(x)

x = Conv2D(128, (5, 5), activation='relu', kernel_regularizer=l2(0.001))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.15)(x)
x = BatchNormalization()(x)

x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(512, (3, 3), activation='relu')(x)
x = Dropout(0.15)(x)

x = Conv2D(1024, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.175)(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

# Output layer: Adjusted to output 8 values (2 for gaze, 6 for head pose)
output_layer = Dense(8, activation='sigmoid')(x)

# Model definition
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])


In [20]:
checkpoint = ModelCheckpoint('eye_gaze_v25_v1.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# EarlyStopping to stop training when the validation loss has not improved after 10 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

callbacks_list = [checkpoint, early_stopping]

In [21]:
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Split indices into training and validation sets
train_indices = indices[:int(0.85 * num_samples)]  # 85% for training
val_indices = indices[int(0.85 * num_samples):]  # 15% for validation

# Instantiate the generators
train_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, train_indices, shuffle=True)
validation_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, val_indices, shuffle=False)

# Calculate steps
steps_per_epoch = len(train_indices) // batch_size
validation_steps = len(val_indices) // batch_size

# Train the model
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks_list,
    verbose=1
)

Epoch 1/100
 372/5780 [>.............................] - ETA: 21:52 - loss: 0.0899 - mean_squared_error: 0.0248 - mean_absolute_error: 0.1100

In [5]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.regularizers import l2
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization

# Your model definition
model = Sequential([
    Conv2D(32, (7, 7), activation='relu', input_shape=(100, 200, 3), kernel_regularizer=l2(0.001)),
    
    
    Conv2D(64, (7, 7), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.15),
 

    Conv2D(128, (5, 5), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D((2, 2)),
    Dropout(0.15),
    BatchNormalization(),
    

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    Dropout(0.15),

    Conv2D(1024, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.175),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='sigmoid')
])


model.compile(optimizer= Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the checkpoint callback to save every 5 epochs
checkpoint = ModelCheckpoint('eye_gaze_v23_2_{epoch:02d}.h5', save_freq=5*steps_per_epoch)


In [ ]:
# Assuming you have already set up your model

# Create generators
train_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, shuffle=True)
validation_generator = memmap_batch_generator(x_memmap_path, y_memmap_path, batch_size, shuffle=False)  # Assuming you can use the same for simplicity


# Train the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # Adjust as needed
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, checkpoint]
)

In [3]:
#load the model
from keras.models import load_model
model = load_model('./models/eye_gaze_v23V99.h5')

In [4]:
# Example: Evaluate on a test set
x_test_memmap = np.memmap('x_dataset.memmap', dtype=x_dtype, mode='r', shape=(num_samples,) + x_shape)
y_test_memmap = np.memmap('y_dataset.memmap', dtype=y_dtype, mode='r', shape=(num_samples,) + y_shape)

# Assuming the last 15% of the data is for testing
test_start_index = int(num_samples * 0.99)
x_test = x_test_memmap[test_start_index:]
y_test = y_test_memmap[test_start_index:]

model.evaluate(x_test, y_test, batch_size=16)


137/137 [==============================] - 7s 13ms/step - loss: 0.0051 - mean_squared_error: 0.0034 - mean_absolute_error: 0.0425


[0.005079503171145916, 0.003443555673584342, 0.04253864660859108]

In [9]:
model.save('./models/eye_gaze_v23v99.h5')